## Core MSTICPy initialization for Notebooks

In [1]:
from msticpy.nbtools import nbinit
nbinit.init_notebook(namespace=globals())
qry_prov = QueryProvider("AzureSentinel")

Please wait. Loading Kqlmagic extension...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [2]:
qry_prov.connect(WorkspaceConfig())
tables = qry_prov.schema_tables

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Choose A Table To Analzye

In [3]:
import ipywidgets as widgets
from IPython.display import display

print('\nPlease select a table to analyze.\n')
tableDropdown = widgets.Dropdown(
    options=sorted(tables),
    value=sorted(tables)[0],
    description='Table:',
)
display(tableDropdown)


Please select a table to analyze.



Dropdown(description='Table:', options=('AACAudit', 'AACHttpRequest', 'AADDomainServicesAccountLogon', 'AADDom…

## Choose a Feature(s) and Timeframe to Analyze

In [9]:
df = qry_prov.exec_query(f'{tableDropdown.value} | take 1')
timestamp_col = 'TimeGenerated'

if df.empty:
    print("Table is empty, please select another table.")
else:
    print ("\nWhat kind of features/columns would you like to see be analyzed?\n")
    options = sorted(list(df))
    options.remove(timestamp_col)
    try:
        options.remove('Type')
    except TypeError:
        print('Type does not exist')
    selected_features = nbwidgets.SelectSubset(source_items=options)
    print('\n\nWhat time frame do you want to analyze?\n')
    timeFrame = nbwidgets.QueryTime(units='day', max_before=20, before=5, max_after=1)
    timeFrame.display()

<IPython.core.display.Javascript object>


What kind of features/columns would you like to see be analyzed?





What time frame do you want to analyze?



## Generate Timeseries Anomalies

In [15]:
def mapAnomalousColors(cells):
    if cells.name in anomalousFeatures:
        return ['background-color: #ffcccb' for i in range(len(cells))]
    return ['background-color: #90EE90' for i in range(len(cells))]

In [16]:
from time_series_utils import check_kwargs, ts_anomalies_stl
from datetime import timedelta

start = (timeFrame.start)
end = (timeFrame.end)
features = selected_features.selected_values

featureDict = {}
anomalyDict = {}
timeframeDict = {}

if start == end:
    print("\nPlease make sure the start and end date are distinct\n")
elif len(features) == 0:
    print("\nPlease choose at least one feature\n")
else:
    for feature in features:
        raw_times_series_data = qry_prov.MultiDataSource.get_timeseries_data(
            start=start,
            end=end,
            table=f"{tableDropdown.value}",
            timestampcolumn=timestamp_col,
            aggregatecolumn=f"{feature}",
            aggregatefunction=f"dcount(tostring({feature}))",
            add_query_items=f'|mv-expand {timestamp_col} to typeof(datetime), {feature} to typeof(long)',
        )
        df_time_series = raw_times_series_data[[timestamp_col, f'{feature}']]
        df_time_series = df_time_series.set_index(timestamp_col)
        anomalies = ts_anomalies_stl(df_time_series)
        featureDict[f'{feature}'] = anomalies
        anomalous_timestamps = list(anomalies[anomalies['anomalies']==1][timestamp_col])
        for timestamp in anomalous_timestamps:
            if timestamp not in anomalyDict:
                anomalyDict[timestamp] = [f'{feature}']
            else:
                anomalyDict[timestamp].append(f'{feature}')
        print(f'Timeseries for {feature} generated')
    print('\nQuerying Raw Data For ')
    print('\nTimestamp for Anomalous Features\n')
    anomalyDf = pd.DataFrame(anomalyDict.items(), columns=[timestamp_col, 'Anomalous Features'])
    anomalyDf = anomalyDf.sort_values(by=[timestamp_col]).reset_index(drop=True)
    anomalyTimeStamps = list(anomalyDf[timestamp_col])
    for timestamp in anomalyTimeStamps:
        # Timerange at which the raw data is retrieved. Default 1 hour before and 2 hours (including the 1 hour time range)
        startRange = timestamp - timedelta(hours=1)
        endRange = timestamp + timedelta(hours=2)
        anomalousFeatures = list(anomalyDf.loc[anomalyDf[timestamp_col] == timestamp]['Anomalous Features'])[0]
        queryString = f"{tableDropdown.value} | where TimeGenerated between(datetime({startRange})..datetime({endRange}))"
        result = qry_prov.exec_query(queryString)
        result = result.style.apply(mapAnomalousColors)
        timeframeDict[timestamp] = result
    display(anomalyDf)

<IPython.core.display.Javascript object>

Timeseries for ClientAppId generated


<IPython.core.display.Javascript object>

Timeseries for ClientIP generated

Querying Raw Data For 

Timestamp for Anomalous Features



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,TimeGenerated,Anomalous Features
0,2021-08-15 10:44:56.110052+00:00,[ClientAppId]
1,2021-08-18 21:44:56.110052+00:00,[ClientAppId]


## Select A Timestamp To Visualize

In [17]:
try:
    timestamp_dropdown = widgets.Dropdown(
        options=sorted(list(anomalyDf[timestamp_col])),
        description='TimeStamp:',
        disabled=False,
    )
    print('\nSelect a timestamp to visualize anomalous features ')
    display(timestamp_dropdown)
except NameError:
    print("Anomaly Dataframe not instantiated yet.")


Select a timestamp to visualize anomalous features 


Dropdown(description='TimeStamp:', options=(Timestamp('2021-08-15 10:44:56.110052+0000', tz='UTC'), Timestamp(…

## Visualize Anomalies

In [18]:
from msticpy.nbtools.timeseries import display_timeseries_anomolies
pd.set_option("display.max_rows", None, "display.max_columns", None)

try:
    anomalous_features = tuple(anomalyDf.loc[anomalyDf[timestamp_col] == timestamp_dropdown.value]['Anomalous Features'])[0]
    for feature in anomalous_features:
        print(f"\nTime Series for {feature}\n")
        display_timeseries_anomolies(data=featureDict[feature], y=feature)
    display(timeframeDict[timestamp_dropdown.value])
except NameError:
    print("Anomaly Dataframe not instantiated yet.")


Time Series for ClientAppId



Loading BokehJS ...

,TenantId,Application,UserDomain,UserAgent,RecordType,TimeGenerated,Operation,OrganizationId,OrganizationId_,UserType,UserKey,OfficeWorkload,ResultStatus,ResultReasonType,OfficeObjectId,UserId,UserId_,ClientIP,ClientIP_,Scope,Site_,ItemType,EventSource,Source_Name,MachineDomainInfo,MachineId,Site_Url,Site_Url_,SourceRelativeUrl,SourceRelativeUrl_,SourceFileName,SourceFileName_,SourceFileExtension,DestinationRelativeUrl,DestinationFileName,DestinationFileExtension,UserSharedWith,SharingType,CustomEvent,Event_Data,ModifiedObjectResolvedName,Parameters,ExternalAccess,OriginatingServer,OrganizationName,Logon_Type,InternalLogonType,MailboxGuid,MailboxOwnerUPN,MailboxOwnerSid,MailboxOwnerMasterAccountSid,LogonUserSid,LogonUserDisplayName,ClientInfoString,Client_IPAddress,ClientMachineName,ClientProcessName,ClientVersion,Folder,CrossMailboxOperations,DestMailboxId,DestMailboxOwnerUPN,DestMailboxOwnerSid,DestMailboxOwnerMasterAccountSid,DestFolder,Folders,AffectedItems,Item,ModifiedProperties,SendAsUserSmtp,SendAsUserMailboxGuid,SendOnBehalfOfUserSmtp,SendonBehalfOfUserMailboxGuid,ExtendedProperties,Client,LoginStatus,Actor,ActorContextId,ActorIpAddress,InterSystemsId,IntraSystemId,SupportTicketId,TargetContextId,DataCenterSecurityEventType,EffectiveOrganization,ElevationTime,ElevationApprover,ElevationApprovedTime,ElevationRequestId,ElevationRole,ElevationDuration,GenericInfo,SourceSystem,OfficeId,SourceRecordId,AzureActiveDirectory_EventType,AADTarget,Start_Time,OfficeTenantId,OfficeTenantId_,TargetUserOrGroupName,TargetUserOrGroupType,MessageId,Members,TeamName,TeamGuid,ChannelType,ChannelName,ChannelGuid,ExtraProperties,AddOnType,AddonName,TabType,Name,OldValue,NewValue,ItemName,ChatThreadId,ChatName,CommunicationType,AADGroupId,AddOnGuid,AppDistributionMode,TargetUserId,OperationScope,AzureADAppId,OperationProperties,AppId,ClientAppId,Type,_ResourceId
0,8ecf8077-cf51-4820-aadd-14040956f35d,,,,50,2021-08-15 10:10:22+00:00,MailItemsAccessed,4b2462a4-bbee-495a-a0e1-f23ae524cc9c,4b2462a4-bbee-495a-a0e1-f23ae524cc9c,Regular,100320003F88D275,Exchange,Succeeded,Succeeded,,seb@seccxpninja.onmicrosoft.com,seb@seccxpninja.onmicrosoft.com,,,,,,,,,,,,,,,,,,,,,,,,,,False,DM8P221MB0375 (15.20.4200.000),seccxpninja.onmicrosoft.com,Owner,0.000000,0c7658e4-8257-4b5d-ba2a-ca7ca9754713,seb@seccxpninja.onmicrosoft.com,S-1-5-21-1215315387-180893480-700816437-7820455,,S-1-5-21-1215315387-180893480-700816437-7820455,,Client=REST;;,40.91.208.65,,,,,None,,,,,,"[{""FolderItems"":[{""InternetMessageId"":""""}],""Id"":""LgAAAAAtsuHHaTeQQL+uUjEybjuMAQDTSDCO3pbrSp9JHdD7qFOIAAAAAAEMAAAB"",""Path"":""\\Inbox""}]",,,,,,,,,,nan,,,,,,,,nan,,2021-08-15 10:20:23+00:00,,NaT,,,nan,,OfficeActivityManager,757b8033-a2bd-4c6d-b92c-70c347e37017,757b8033-a2bd-4c6d-b92c-70c347e37017,,,2021-08-15 10:20:23+00:00,$RestApiTenantId$,$RestApiTenantId$,,,,None,,,,,,None,,,,,,,,,,,,,,,,,"[{'Value': 'Bind', 'Name': 'MailAccessType'}, {'Value': 'False', 'Name': 'IsThrottled'}]",7ab7862c-4c57-491e-8a45-d52a7e023983,,OfficeActivity,
1,8ecf8077-cf51-4820-aadd-14040956f35d,,,,ExchangeAdmin,2021-08-15 09:53:37+00:00,Set-User,4b2462a4-bbee-495a-a0e1-f23ae524cc9c,4b2462a4-bbee-495a-a0e1-f23ae524cc9c,DcAdmin,NT AUTHORITY\SYSTEM (Microsoft.Exchange.Management.ForwardSync),Exchange,True,True,NAMPR06A007.PROD.OUTLOOK.COM/Microsoft Exchange Hosted Organizations/seccxpninja.onmicrosoft.com/ThreatIntel,NT AUTHORITY\SYSTEM (Microsoft.Exchange.Management.ForwardSync),NT AUTHORITY\SYSTEM (Microsoft.Exchange.Management.ForwardSync),,,,,,,,,,,,,,,,,,,,,,,,,"[{""Name"":""Identity"",""Value"":""4b2462a4-bbee-495a-a0e1-f23ae524cc9c\\ccec4856-3259-4631-a860-eabd023c6106""},{""Name"":""SyncMailboxLocationGuids"",""Value"":""True""},{""Name"":""ErrorAction"",""Value"":""Stop""},{""Name"":""WarningAction"",""Value"":""SilentlyContinue""}]",True,DM8P221MB0601 (15.20.4415.022),seccxpninja.onmicrosoft.com,,nan,,,,,,,,,,,,,None,,,,,,,,,,,,,,,,nan,,,,,,,,nan,,2021-08-15 10:00:24+00:00,,NaT,,,nan,,OfficeActivityMana